In [1]:
from dotenv import load_dotenv

load_dotenv()
from falkordb_gemini_kg.classes.model_config import KnowledgeGraphModelConfig
from falkordb_gemini_kg.models.openai import OpenAiGenerativeModel
from falkordb_gemini_kg import KnowledgeGraph, Ontology
from falkordb_gemini_kg.classes.source import URL
import vertexai
import os
from random import shuffle
import json
from falkordb import FalkorDB

# Initialize the Vertex AI client
vertexai.init(project=os.getenv("PROJECT_ID"), location=os.getenv("REGION"))

### Import source data

In [2]:
src_file="../data/movies/rottentomatoes.txt"

source_urls = []
with open(src_file, "r", encoding="utf-8") as file:
    source_urls = file.readlines()

shuffle(source_urls)

sources = [URL(url) for url in source_urls]

model = OpenAiGenerativeModel(model_name="gpt-4o")

### Automatically create the ontology from 20% of the sources

In [ ]:
boundaries = "Extract all information related to the movies. The graph does not need to contain data about the plot of the movie, but everything else related to it."

ontology = Ontology.from_sources(
    sources=sources[: round(len(sources) * 0.2)],
    boundaries=boundaries,
    model=model,
)


db = FalkorDB()
graph = db.select_graph("movies_ontology")
ontology.save_to_graph(graph)

# Save ontology to json file
with open("ontology_n.json", "w", encoding="utf-8") as file:
    file.write(json.dumps(ontology.to_json(), indent=2))

### Read ontology from json file

In [3]:
ontology_file = "ontology.json"
with open(ontology_file, "r", encoding="utf-8") as file:
    ontology = Ontology.from_json(json.loads(file.read()))

kg = KnowledgeGraph(
    name="movies",
    model_config=KnowledgeGraphModelConfig.with_model(model),
    ontology=ontology,
)

### Process the sources raw data into the knowledge graph

In [ ]:
kg.process_sources(sources)

Relation with label MENTIONED not found in ontology


### Ask a single question to the model

In [ ]:
kg.ask("How are Keanu Reeves and Carrie-Anne Moss related?")

'Keanu Reeves and Carrie-Anne Moss have acted in the same movies, "The Matrix" and "The Matrix Reloaded." \n'

### Start a chat session with the model

In [7]:
chat = kg.chat_session()

print(chat.send_message("Who is the director of the movie The Matrix?"))
print(chat.send_message("And how are they related with Keanu Reeves?"))

Lana Wachowski and Lilly Wachowski directed the movie The Matrix. 

Lana Wachowski and Lilly Wachowski directed the movie The Matrix, in which Keanu Reeves acted. 

